In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD
from sklearn.utils.extmath import randomized_svd
from sklearn.datasets import fetch_openml
from oja_pca import OjaPCA
import torch

from svd import ApproxSVD 

In [2]:
try:
    import fbpca
    HAS_FBPCA = True
except ImportError:
    HAS_FBPCA = False

In [3]:
def load_mnist_subset(n_samples=5000):
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X = mnist.data.astype(np.float32).T[:, :n_samples] / 255.0
    return X  # shape: (784, n_samples)

def load_fashion_mnist_subset(n_samples=5000):
    fmnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
    X = fmnist.data.astype(np.float32).T[:, :n_samples] / 255.0
    return X  # shape: (784, n_samples)

def load_usps_subset(n_samples=5000):
    usps = fetch_openml('USPS', version=1, as_frame=False)
    X = usps.data.astype(np.float32).T[:, :n_samples] / 255.0
    return X  # shape: (256, n_samples) since USPS has 16x16 images

def load_isolet_subset(n_samples=5000):
    isolet = fetch_openml('isolet', version=1, as_frame=False)
    # Features are already real-valued, just normalize by max
    X = isolet.data.astype(np.float32).T[:, :n_samples]
    X /= np.max(X)  # scale to [0,1]
    return X  # shape: (617, n_samples), 617 audio features

def load_mnist_and_fashion(n_samples=5000):
    # Load MNIST
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X_mnist = mnist.data.astype(np.float32)[:n_samples] / 255.0
    y_mnist = mnist.target.astype(int)[:n_samples]

    # Load Fashion-MNIST
    fmnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
    X_fmnist = fmnist.data.astype(np.float32)[:n_samples] / 255.0
    y_fmnist = fmnist.target.astype(int)[:n_samples]

    # Stack them together
    X = np.vstack([X_mnist, X_fmnist]).T   # shape: (784, 2*n_samples)
    return X

In [4]:
def explained_variance_ratio(X, X_recon):
    print(X.shape)
    print(X_recon.shape)
    error = np.linalg.norm(X - X_recon, 'fro') ** 2
    total = np.linalg.norm(X, 'fro') ** 2
    return 1 - error / total

In [5]:
def benchmark(method_name, fit_fn):
    start = time.time()
    U, S, Vt, X_recon = fit_fn()
    elapsed = time.time() - start
    evr = explained_variance_ratio(X, X_recon)
    return {
        "method": method_name,
        "time": elapsed,
        "explained_variance": evr,
    }

In [6]:
def run_benchmarks(X, p=50, g=200):
    results = []

    ApproxSVD
    def run_approx():
        approx_svd = ApproxSVD(n_iter=g, p=p,
                               score_method="cf",
                               debug_mode=True,
                               jobs=8,
                               stored_g = False,
                               use_shared_memory=False,
                               use_heap="optimized_heap")
        _, U, X_approx = approx_svd.fit_batched(X, 100000)
        X_reduced = U.T[:p, :] @ X
        X_recon = U[:, :p] @ X_reduced
        return U, None, None, X_recon
    results.append(benchmark("ApproxSVD", run_approx))

    # sklearn PCA (full SVD)
    def run_pca():
        model = PCA(n_components=p, svd_solver="full")
        model.fit(X.T)
        X_recon = model.inverse_transform(model.transform(X.T)).T
        return model.components_.T, model.singular_values_, None, X_recon
    results.append(benchmark("PCA (full)", run_pca))

    # Incremental PCA
    def run_incpca():
        model = IncrementalPCA(n_components=p, batch_size=100000)
        model.fit(X.T)
        X_recon = model.inverse_transform(model.transform(X.T)).T
        return model.components_.T, None, None, X_recon
    results.append(benchmark("IncrementalPCA", run_incpca))

     #TruncatedSVD (randomized)
    def run_tsvd():
        model = TruncatedSVD(n_components=p)
        X_reduced = model.fit_transform(X.T)
        # Reconstruction: approximate, since TSVD doesn't store mean
        X_recon = (X_reduced @ model.components_).T
        return model.components_.T, None, None, X_recon
    results.append(benchmark("TruncatedSVD", run_tsvd))

    def run_oja():
        model = OjaPCA(
            n_features=X.shape[0],
            n_components=p,
            eta=0.005,
        )
        X_tensor = torch.tensor(X.T)
        b_size = 100000
        for i in range(0, len(X_tensor) - b_size, b_size):
            batch = X_tensor[i : i + b_size]
            if len(batch) < b_size:
                # This line means we use up to an extra partial batch over 1 pass
                batch = torch.cat([batch, X_tensor[: b_size - len(batch)]], dim=0)
            error = model(batch) if hasattr(model, "forward") else None
        recon = model.inverse_transform(model.transform(X_tensor))
        return np.array(model.get_components()), None, None, np.array(recon).T
    results.append(benchmark("OjaPCA", run_oja))

    # fbpca (if available)
    if HAS_FBPCA:
        def run_fbpca():
            U, s, Vt = fbpca.pca(X, k=p, raw=True)
            X_recon = (U[:, :p] * s[:p]) @ Vt[:p, :]
            return U, s, Vt, X_recon
        results.append(benchmark("fBPCA", run_fbpca))

    return results

In [ ]:
X = load_mnist_and_fashion(n_samples=60000)
# X = np.random.rand(784,300000).astype(np.float32)
n_samples = 600000
repeats = int(np.ceil(n_samples / X.shape[1]))
X = np.tile(X, (1, repeats))[:, :n_samples]
# np.random.seed(42) 
# X = np.random.rand(5, 8)
p = 200
g = 10000

results = run_benchmarks(X, p=p, g=g)

print("\nBenchmark Results:")
for r in results:
    print(f"{r['method']:15s} | Time: {r['time']:.2f}s | Explained Var: {r['explained_variance']*100:.2f}%")

# Optional: bar plot
methods = [r["method"] for r in results]
times = [r["time"] for r in results]
evrs = [r["explained_variance"] for r in results]

fig, ax1 = plt.subplots(figsize=(8,4))
ax2 = ax1.twinx()

ax1.bar(methods, times, alpha=0.6, label="Time (s)")
ax2.plot(methods, evrs, "o-", color="red", label="Explained Var")

ax1.set_ylabel("Time (s)")
ax2.set_ylabel("Explained Variance")
plt.title(f"PCA Benchmark (p={p}, g={g}) on MNIST subset")
plt.show()

  0%|          | 0/10000 [00:00<?, ?it/s]d:\online_pca\svd.py:263: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "svd.py", line 96:
@njit(parallel=True)
def compute_score_cf_numba(i, j, x, d):
^

  new_vals[s] = compute_score_cf_numba(iq, s, x, d)[2]
100%|██████████| 10000/10000 [00:51<00:00, 192.50it/s]


{'initial scores': {'total': 2.9325246999997034, 'count': 1, 'avg': 2.9325246999997034}, 'build heap': {'total': 0.3369719000002078, 'count': 1, 'avg': 0.3369719000002078}, 'perform small svd': {'total': 1.001683199991021, 'count': 10000, 'avg': 0.0001001683199991021}, 'perform matrix mul': {'total': 22.656983899934858, 'count': 10000, 'avg': 0.002265698389993486}, 'calculate row score': {'total': 3.5572049999100273, 'count': 10253, 'avg': 0.0003469428459875185}, 'rebuild heap': {'total': 3.9407712000393076, 'count': 10253, 'avg': 0.00038435298937279893}, 'update cols': {'total': 19.5495273998622, 'count': 10000, 'avg': 0.00195495273998622}, 'total time': {'total': 55.23344030000044, 'count': 1, 'avg': 55.23344030000044}}


  0%|          | 0/10000 [00:00<?, ?it/s]d:\online_pca\svd.py:263: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "svd.py", line 96:
@njit(parallel=True)
def compute_score_cf_numba(i, j, x, d):
^

  new_vals[s] = compute_score_cf_numba(iq, s, x, d)[2]
100%|██████████| 10000/10000 [00:38<00:00, 258.56it/s]


{'initial scores': {'total': 1.5277626999995846, 'count': 1, 'avg': 1.5277626999995846}, 'build heap': {'total': 0.11830000000009022, 'count': 1, 'avg': 0.11830000000009022}, 'perform small svd': {'total': 1.097901399996772, 'count': 10000, 'avg': 0.00010979013999967719}, 'perform matrix mul': {'total': 10.332526700016388, 'count': 10000, 'avg': 0.0010332526700016387}, 'calculate row score': {'total': 3.753846299883662, 'count': 10633, 'avg': 0.000353037364796733}, 'rebuild heap': {'total': 4.043215299820076, 'count': 10633, 'avg': 0.00038025160348162103}, 'update cols': {'total': 18.374999400059096, 'count': 10000, 'avg': 0.0018374999400059095}, 'total time': {'total': 40.321128400000816, 'count': 1, 'avg': 40.321128400000816}}


  3%|▎         | 327/10000 [00:02<00:51, 188.32it/s]